## Cafile

In [8]:
import ssl

import httpx
import trustme

# Look, you just created your certificate authority!
ca = trustme.CA()

# And now you issued a cert signed by this fake CA
# https://en.wikipedia.org/wiki/Example.org
server_cert = ca.issue_cert("test-host.example.org")

# That's it!

# ----- Using your shiny new certs -----
# Use an explicitly configured certificate store.
ctx = ssl.create_default_context(cafile="client.pem")  # Either cafile or capath.
ca.configure_trust(ctx)
client = httpx.Client(verify=ctx)
# ----- Creating certs -----
# You can configure SSL context objects to trust this CA:
# Or configure them to present the server certificate
# server_cert.configure_cert(ssl_context)
resp = client.get("https://localhost:8000")
print(resp.text)

Hello, world!


## mTLS

In [15]:
import ssl

import httpx

# Paths to your certificates
CERT_DIR = "tests/certs"
CA_CERT = f"{CERT_DIR}/ca.pem"  # CA certificate (used to verify server)
CLIENT_CERT = f"{CERT_DIR}/client.pem"  # Client certificate (used for mTLS)
CLIENT_KEY = f"{CERT_DIR}/client.key"  # Client private key (used for mTLS)

# Configure SSL context for mutual TLS
ctx = ssl.create_default_context(cafile=CA_CERT)  # Trust the CA
ctx.load_cert_chain(certfile=CLIENT_CERT, keyfile=CLIENT_KEY)  # Client auth

# Create an HTTPX client with mTLS
with httpx.Client(verify=ctx) as client:
    response = client.get("https://localhost:8000/")
    print(response.text)

Hello, world!
